In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd



import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow import keras
from operations import adjust_dataset, split_data



2023-12-02 10:43:36.724925: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-02 10:43:36.728932: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-02 10:43:36.789937: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 10:43:36.789970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 10:43:36.790719: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

# Implementation of ANN

In [36]:

df = pd.read_csv('docs/data/GlobalWeatherRepository.csv')

df = adjust_dataset(df, 'Madrid')


In [37]:
possibilities = ['temperature_celsius', 'wind_kph', 'humidity', 'pressure_mb', 'precip_mm', 'cloud']


X_train, y_train, X_test, y_test = split_data(df, 'temperature_celsius', int(len(df) * 0.7))

# print(X_train.shape)

In [38]:
# Lets build the model
# We will use a simple neural network with 3 layers and a sliding window of 7 days and 3 days as output


# We will use a sliding window of 7 days and 3 days as output
window_size = 7
output_size = 3

def create_sequences_and_labels(data, window_size, output_size):
    sequences, labels = [], []
    for i in range(len(data) - window_size - output_size + 1):
        seq = data[i:i + window_size]
        label = data[i + window_size:i + window_size + output_size]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# Create sequences and labels for training data
X_train_seq, y_train_seq = create_sequences_and_labels(y_train, window_size, output_size)

# Create sequences and labels for test data
X_test_seq, y_test_seq = create_sequences_and_labels(y_test, window_size, output_size)

# Define the model
model = keras.Sequential([
    keras.layers.Dense(32, input_shape=(window_size,), activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(output_size, activation='linear')
])

# Compilation of model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

# Train the model
model.fit(X_train_seq, y_train_seq, epochs=100)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(X_test_seq, y_test_seq, verbose=2)

print('Test mean squared error:', test_acc)


Epoch 1/100
2/2 [==============================] - 1s 9ms/step - loss: 311.1078 - mean_squared_error: 311.1078
Epoch 2/100
2/2 [==============================] - 0s 15ms/step - loss: 269.1266 - mean_squared_error: 269.1266
Epoch 3/100
2/2 [==============================] - 0s 14ms/step - loss: 230.3842 - mean_squared_error: 230.3842
Epoch 4/100
2/2 [==============================] - 0s 8ms/step - loss: 195.0840 - mean_squared_error: 195.0840
Epoch 5/100
2/2 [==============================] - 0s 9ms/step - loss: 162.9996 - mean_squared_error: 162.9996
Epoch 6/100
2/2 [==============================] - 0s 14ms/step - loss: 134.6483 - mean_squared_error: 134.6483
Epoch 7/100
2/2 [==============================] - 0s 9ms/step - loss: 109.5826 - mean_squared_error: 109.5826
Epoch 8/100
2/2 [==============================] - 0s 6ms/step - loss: 87.6990 - mean_squared_error: 87.6990
Epoch 9/100
2/2 [==============================] - 0s 8ms/step - loss: 69.7643 - mean_squared_error: 69.7643
Ep

In [39]:
# Now, we will predict the temperature for the next 3 days

# Predict the temperature for the next 3 days
predictions = model.predict(X_test_seq[-1].reshape(1, window_size))

print('Predicted temperature for the next 3 days:', predictions[0])

1/1 [==============================] - 0s 65ms/step
Predicted temperature for the next 3 days: [9.881827  9.3954735 7.9645042]
